Week 2 Peer Graded Assessment (Battle of Neighbourhoods)

Note to Peers: I have completed the entire assignment for week 3 in this single notebook. I will provide the link to this notebook in all the required fields. Please evaluate accordingly.

# Part 1: Data wrangling and pre-processing

In [1]:
import numpy as np
import pandas as pd
import json
from sklearn.cluster import KMeans
import folium
import requests
from geopy.geocoders import Nominatim
from pandas.io.json import json_normalize

In [2]:
source = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
result = requests.get(source)

result.headers

{'Date': 'Wed, 27 May 2020 04:43:48 GMT', 'Vary': 'Accept-Encoding,Cookie,Authorization', 'Server': 'ATS/8.0.7', 'Content-Type': 'text/html; charset=UTF-8', 'X-Content-Type-Options': 'nosniff', 'P3P': 'CP="See https://en.wikipedia.org/wiki/Special:CentralAutoLogin/P3P for more info."', 'Content-language': 'en', 'Last-Modified': 'Sat, 23 May 2020 19:32:34 GMT', 'Content-Encoding': 'gzip', 'Age': '33332', 'X-Cache': 'cp5008 hit, cp5011 hit/38', 'X-Cache-Status': 'hit-front', 'Server-Timing': 'cache;desc="hit-front"', 'Strict-Transport-Security': 'max-age=106384710; includeSubDomains; preload', 'Set-Cookie': 'WMF-Last-Access=27-May-2020;Path=/;HttpOnly;secure;Expires=Sun, 28 Jun 2020 12:00:00 GMT, WMF-Last-Access-Global=27-May-2020;Path=/;Domain=.wikipedia.org;HttpOnly;secure;Expires=Sun, 28 Jun 2020 12:00:00 GMT, GeoIP=IN:DL:New_Delhi:28.63:77.22:v4; Path=/; secure; Domain=.wikipedia.org', 'X-Client-IP': '125.99.211.184', 'Cache-Control': 'private, s-maxage=0, max-age=0, must-revalidate'

In [3]:
from bs4 import BeautifulSoup
bs = BeautifulSoup(result.content, 'html.parser')
table = bs.find('table')
trs = table.find_all('tr')
rows = []

for tr in trs:
    i = tr.find_all('td')
    if i:
        rows.append(i)

lst = []
for row in rows:
    postalcode = row[0].text.rstrip()
    borough = row[1].text.rstrip()
    neighborhood = row[2].text.rstrip()
    if borough != 'Not assigned':
        if neighborhood == 'Not assigned':
            neighborhood = borough
        lst.append([postalcode, borough, neighborhood])
        
lst        

[['M3A', 'North York', 'Parkwoods'],
 ['M4A', 'North York', 'Victoria Village'],
 ['M5A', 'Downtown Toronto', 'Regent Park, Harbourfront'],
 ['M6A', 'North York', 'Lawrence Manor, Lawrence Heights'],
 ['M7A', 'Downtown Toronto', "Queen's Park, Ontario Provincial Government"],
 ['M9A', 'Etobicoke', 'Islington Avenue, Humber Valley Village'],
 ['M1B', 'Scarborough', 'Malvern, Rouge'],
 ['M3B', 'North York', 'Don Mills'],
 ['M4B', 'East York', 'Parkview Hill, Woodbine Gardens'],
 ['M5B', 'Downtown Toronto', 'Garden District, Ryerson'],
 ['M6B', 'North York', 'Glencairn'],
 ['M9B',
  'Etobicoke',
  'West Deane Park, Princess Gardens, Martin Grove, Islington, Cloverdale'],
 ['M1C', 'Scarborough', 'Rouge Hill, Port Union, Highland Creek'],
 ['M3C', 'North York', 'Don Mills'],
 ['M4C', 'East York', 'Woodbine Heights'],
 ['M5C', 'Downtown Toronto', 'St. James Town'],
 ['M6C', 'York', 'Humewood-Cedarvale'],
 ['M9C',
  'Etobicoke',
  'Eringate, Bloordale Gardens, Old Burnhamthorpe, Markland Wood

Finally, printing the shape of the Data Frame and checking if merging  of neighborhoods was successful

In [4]:
cols = ['Postal Code', 'Borough', 'Neighborhood']
df = pd.DataFrame(lst, columns = cols)
print(df.shape)
df.loc[df['Postal Code'] == 'M5A']

(103, 3)


,Postal Code,Borough,Neighborhood
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"


# Part 2: Using csv file to add co-ordinates to the Data Frame

In [5]:
df_coor = pd.read_csv(r"C:\Users\Raevant\Downloads\Geospatial_Coordinates.csv")
df_fin = pd.merge(df, df_coor, on = "Postal Code", how = 'left')
df_fin.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


# Part 3: Toronto Map, Cluster Analysis

In [6]:
geolocator = Nominatim(user_agent = "coursera")
address = 'Toronto'

try:
    location = geolocator.geocode(address)
    latitude = location.latitude
    longitude = location.longitude
    print('The geograpical coordinates of {} are {}, {}.'.format(address, latitude, longitude))
except AttributeError:
    print('Cannot find: {}, will drop index: {}'.format(address, index))
    
tor_map = folium.Map(location = [latitude, longitude], zoom_start = 11)

for lat, lon, label in zip(df_fin['Latitude'], df_fin['Longitude'], df_fin['Postal Code']):
    label = folium.Popup(label)
    folium.CircleMarker([lat, lon], raduis = 2, popup = label, color = 'blue', fill_color='#3186cc',
        fill_opacity=0.7).add_to(tor_map)
    
tor_map    

The geograpical coordinates of Toronto are 43.6534817, -79.3839347.


## Final Analysis begins...

In [7]:
CLIENT_ID = '2EMPKKWRACCMBJV2QIDK15TUR2CMBJEXYYHK2FPXNDEB1BH5'
CLIENT_SECRET = 'BTOIKORMJ2UUMBUGJYC5LYANUD2RG52PONHEIPSGFGYR4HVY'
VERSION = '20200527'

#### My focus will be on Scarborough (M1P)

In [8]:
pd.set_option('display.max_rows', None)
df_fin

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village",43.667856,-79.532242
6,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
7,M3B,North York,Don Mills,43.745906,-79.352188
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937


In [9]:
LIMIT = 90
Radius = 500
df_fin.set_index('Postal Code', inplace = True)
neighbor_lat = df_fin.loc['M1P']['Latitude']
neighbor_lon = df_fin.loc['M1P']['Longitude']

#establishing url
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighbor_lat, 
    neighbor_lon, 
    Radius, 
    LIMIT)

url

'https://api.foursquare.com/v2/venues/explore?&client_id=2EMPKKWRACCMBJV2QIDK15TUR2CMBJEXYYHK2FPXNDEB1BH5&client_secret=BTOIKORMJ2UUMBUGJYC5LYANUD2RG52PONHEIPSGFGYR4HVY&v=20200527&ll=43.7574096,-79.27330400000001&radius=500&limit=90'

In [10]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5ece710f71c428001b4ab26b'},
 'response': {'headerLocation': 'Dorset Park',
  'headerFullLocation': 'Dorset Park, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 8,
  'suggestedBounds': {'ne': {'lat': 43.761909604500005,
    'lng': -79.26708530848985},
   'sw': {'lat': 43.7529095955, 'lng': -79.27952269151017}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '5226562611d2cd49d83ef03b',
       'name': 'Kairali',
       'location': {'address': '1210 Kennedy Road',
        'crossStreet': 'Lawrence',
        'lat': 43.754914739291834,
        'lng': -79.27694504646365,
        'labeledLatLngs': [{'label': 'display',
          'lat': 43.754914739291834,
          'lng': -79.27694504646365}],
        'distance': 40

In [12]:
venues = results['response']['groups'][0]['items']
nearby_venues = json_normalize(venues)
nearby_venues

c:\program files (x86)\microsoft visual studio\shared\python37_64\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  


,referralId,reasons.count,reasons.items,venue.id,venue.name,venue.location.address,venue.location.crossStreet,venue.location.lat,venue.location.lng,venue.location.labeledLatLngs,...,venue.location.postalCode,venue.location.cc,venue.location.city,venue.location.state,venue.location.country,venue.location.formattedAddress,venue.categories,venue.photos.count,venue.photos.groups,venue.venuePage.id
0,e-0-5226562611d2cd49d83ef03b-0,0,"[{'summary': 'This spot is popular', 'type': '...",5226562611d2cd49d83ef03b,Kairali,1210 Kennedy Road,Lawrence,43.754915,-79.276945,"[{'label': 'display', 'lat': 43.75491473929183...",...,M1P 2L1,CA,Toronto,ON,Canada,"[1210 Kennedy Road (Lawrence), Toronto ON M1P ...","[{'id': '4bf58dd8d48988d10f941735', 'name': 'I...",0,[],NaN
1,e-0-4b6475aef964a520eab42ae3-1,0,"[{'summary': 'This spot is popular', 'type': '...",4b6475aef964a520eab42ae3,Kim Kim restaurant,1188 Kennedy Rd,NaN,43.753833,-79.276611,"[{'label': 'display', 'lat': 43.75383283792732...",...,M1P 2L1,CA,Scarborough,ON,Canada,"[1188 Kennedy Rd, Scarborough ON M1P 2L1, Canada]","[{'id': '4bf58dd8d48988d145941735', 'name': 'C...",0,[],55701321
2,e-0-4bf96c435317a593a23a017f-2,0,"[{'summary': 'This spot is popular', 'type': '...",4bf96c435317a593a23a017f,Karaikudi Chettinad South Indian Restaurant,1225 Kennedy Rd,at Forbes (Between Lawrence and Ellesmere),43.756042,-79.276276,"[{'label': 'display', 'lat': 43.75604153945313...",...,M1P 4Y1,CA,Toronto,ON,Canada,[1225 Kennedy Rd (at Forbes (Between Lawrence ...,"[{'id': '4bf58dd8d48988d10f941735', 'name': 'I...",0,[],33638214
3,e-0-4bc75a3c6501c9b630433e29-3,0,"[{'summary': 'This spot is popular', 'type': '...",4bc75a3c6501c9b630433e29,Pho Vietnam,1280 Kennedy Road,at Shropshire Drive,43.757770,-79.278572,"[{'label': 'display', 'lat': 43.75777030564718...",...,M1P 2L5,CA,Toronto,ON,Canada,"[1280 Kennedy Road (at Shropshire Drive), Toro...","[{'id': '4bf58dd8d48988d14a941735', 'name': 'V...",0,[],NaN
4,e-0-4b0dae6af964a520ce4d23e3-4,0,"[{'summary': 'This spot is popular', 'type': '...",4b0dae6af964a520ce4d23e3,Big Al's Pet Supercentre,1295 Kennedy Rd.,at Munham Gate.,43.759279,-79.278325,"[{'label': 'display', 'lat': 43.75927894411595...",...,M1P 2L4,CA,Scarborough,ON,Canada,"[1295 Kennedy Rd. (at Munham Gate.), Scarborou...","[{'id': '4bf58dd8d48988d100951735', 'name': 'P...",0,[],NaN
5,e-0-4bda1b6c63c5c9b641c32268-5,0,"[{'summary': 'This spot is popular', 'type': '...",4bda1b6c63c5c9b641c32268,Scarborough LRT,Eglinton Ave. E,NaN,43.756465,-79.272194,"[{'label': 'display', 'lat': 43.75646495188919...",...,NaN,CA,Scarborough,ON,Canada,"[Eglinton Ave. E, Scarborough ON, Canada]","[{'id': '4bf58dd8d48988d1fc931735', 'name': 'L...",0,[],NaN
6,e-0-4bfebef9e584c928402b6d25-6,0,"[{'summary': 'This spot is popular', 'type': '...",4bfebef9e584c928402b6d25,Salvation Army Thrift Store,1225 Kennedy Rd,NaN,43.755782,-79.276208,"[{'label': 'display', 'lat': 43.75578210550947...",...,M1P 2L4,CA,Scarborough,ON,Canada,"[1225 Kennedy Rd, Scarborough ON M1P 2L4, Canada]","[{'id': '4bf58dd8d48988d101951735', 'name': 'T...",0,[],NaN
7,e-0-4d22042e16cfb60c64226661-7,0,"[{'summary': 'This spot is popular', 'type': '...",4d22042e16cfb60c64226661,Blinds To Go,1293 Kennedy Road,NaN,43.757147,-79.278073,"[{'label': 'display', 'lat': 43.75714688528929...",...,M1P 2L4,CA,Scarborough,ON,Canada,"[1293 Kennedy Road, Scarborough ON M1P 2L4, Ca...","[{'id': '4bf58dd8d48988d1f8941735', 'name': 'F...",0,[],NaN


In [14]:
venues_info = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues = nearby_venues.loc[: ,venues_info]

def get_category(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
    
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']
nearby_venues['venue.categories'] = nearby_venues.apply(get_category, axis = 1)

nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]
nearby_venues

,name,categories,lat,lng
0,Kairali,Indian Restaurant,43.754915,-79.276945
1,Kim Kim restaurant,Chinese Restaurant,43.753833,-79.276611
2,Karaikudi Chettinad South Indian Restaurant,Indian Restaurant,43.756042,-79.276276
3,Pho Vietnam,Vietnamese Restaurant,43.757770,-79.278572
4,Big Al's Pet Supercentre,Pet Store,43.759279,-79.278325
5,Scarborough LRT,Light Rail Station,43.756465,-79.272194
6,Salvation Army Thrift Store,Thrift / Vintage Store,43.755782,-79.276208
7,Blinds To Go,Furniture / Home Store,43.757147,-79.278073


In [15]:
print('{} venues were returned by Foursquare'.format(nearby_venues.shape[0]))

8 venues were returned by Foursquare


In [28]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])
    
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [30]:
venues = getNearbyVenues(names=df_fin['Borough'],latitudes=df_fin['Latitude'],longitudes=df_fin['Longitude'])

North York
North York
Downtown Toronto
North York
Downtown Toronto
Etobicoke
Scarborough
North York
East York
Downtown Toronto
North York
Etobicoke
Scarborough
North York
East York
Downtown Toronto
York
Etobicoke
Scarborough
East Toronto
Downtown Toronto
York
Scarborough
East York
Downtown Toronto
Downtown Toronto
Scarborough
North York
North York
East York
Downtown Toronto
West Toronto
Scarborough
North York
North York
East York
Downtown Toronto
West Toronto
Scarborough
North York
North York
East Toronto
Downtown Toronto
West Toronto
Scarborough
North York
North York
East Toronto
Downtown Toronto
North York
North York
Scarborough
North York
North York
East Toronto
North York
York
North York
Scarborough
North York
North York
Central Toronto
Central Toronto
York
York
Scarborough
North York
Central Toronto
Central Toronto
West Toronto
Etobicoke
Scarborough
North York
Central Toronto
Central Toronto
West Toronto
Mississauga
Etobicoke
Scarborough
Central Toronto
Downtown Toronto
West Toron

In [31]:
venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,North York,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
1,North York,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop
2,North York,43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena
3,North York,43.725882,-79.315572,Tim Hortons,43.725517,-79.313103,Coffee Shop
4,North York,43.725882,-79.315572,Portugril,43.725819,-79.312785,Portuguese Restaurant


In [32]:
venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Central Toronto,114,114,114,114,114,114
Downtown Toronto,1160,1160,1160,1160,1160,1160
East Toronto,125,125,125,125,125,125
East York,74,74,74,74,74,74
Etobicoke,77,77,77,77,77,77
Mississauga,12,12,12,12,12,12
North York,243,243,243,243,243,243
Scarborough,94,94,94,94,94,94
West Toronto,159,159,159,159,159,159


In [34]:
print('{} unique categories found'.format(len(venues['Venue Category'].unique())))

272 unique categories found


In [41]:
onehot = pd.get_dummies(venues[['Venue Category']], prefix="", prefix_sep="")
onehot['Neighborhood'] = venues['Neighborhood'] 

onehot.head()

,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [42]:
grouped = onehot.groupby('Neighborhood').mean().reset_index()
grouped

,Neighborhood,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Central Toronto,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.026316,...,0.008772,0.000000,0.000000,0.008772,0.000000,0.000000,0.000000,0.000000,0.000000,0.008772
1,Downtown Toronto,0.00000,0.000862,0.000862,0.000862,0.000862,0.001724,0.002586,0.000862,0.014655,...,0.010345,0.000862,0.000000,0.003448,0.000000,0.006034,0.000000,0.001724,0.000862,0.006034
2,East Toronto,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.024000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.008000,0.000000,0.000000,0.000000,0.016000
3,East York,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.013514,0.000000,0.013514,0.000000,0.000000,0.000000,0.000000,0.013514
4,Etobicoke,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.012987,...,0.000000,0.000000,0.012987,0.000000,0.000000,0.000000,0.000000,0.012987,0.000000,0.000000
5,Mississauga,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.083333,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,North York,0.00823,0.000000,0.004115,0.000000,0.000000,0.000000,0.000000,0.000000,0.008230,...,0.000000,0.004115,0.004115,0.008230,0.000000,0.000000,0.000000,0.000000,0.004115,0.000000
7,Scarborough,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.010638,...,0.000000,0.000000,0.000000,0.010638,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,West Toronto,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.018868,0.000000,0.000000,0.006289,0.000000,0.006289,0.006289,0.000000,0.000000,0.012579
9,York,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.055556,0.000000


In [43]:
num_top_venues = 5

for hood in grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = grouped[grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Central Toronto----
            venue  freq
0     Coffee Shop  0.07
1  Sandwich Place  0.06
2            Café  0.05
3            Park  0.05
4      Restaurant  0.04


----Downtown Toronto----
                 venue  freq
0          Coffee Shop  0.10
1                 Café  0.06
2                Hotel  0.03
3           Restaurant  0.03
4  Japanese Restaurant  0.02


----East Toronto----
                venue  freq
0    Greek Restaurant  0.07
1         Coffee Shop  0.05
2                Café  0.04
3  Italian Restaurant  0.04
4             Brewery  0.04


----East York----
                 venue  freq
0          Coffee Shop  0.07
1          Pizza Place  0.05
2                 Bank  0.05
3  Sporting Goods Shop  0.04
4         Burger Joint  0.04


----Etobicoke----
                  venue  freq
0           Pizza Place  0.10
1        Sandwich Place  0.06
2  Fast Food Restaurant  0.04
3                  Café  0.04
4           Coffee Shop  0.04


----Mississauga----
                       v

In [44]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [45]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = grouped['Neighborhood']

for ind in np.arange(grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Central Toronto,Coffee Shop,Sandwich Place,Café,Park,Restaurant,Pizza Place,Sushi Restaurant,American Restaurant,Gym,Dessert Shop
1,Downtown Toronto,Coffee Shop,Café,Restaurant,Hotel,Japanese Restaurant,Italian Restaurant,Bakery,Park,Seafood Restaurant,Beer Bar
2,East Toronto,Greek Restaurant,Coffee Shop,Italian Restaurant,Café,Brewery,Park,Restaurant,Ice Cream Shop,Bakery,Light Rail Station
3,East York,Coffee Shop,Bank,Pizza Place,Sporting Goods Shop,Pharmacy,Burger Joint,Park,Gym,Furniture / Home Store,Fast Food Restaurant
4,Etobicoke,Pizza Place,Sandwich Place,Grocery Store,Coffee Shop,Fast Food Restaurant,Gym,Liquor Store,Pharmacy,Café,Pet Store
5,Mississauga,Hotel,Coffee Shop,Middle Eastern Restaurant,Gym,Fried Chicken Joint,Intersection,American Restaurant,Sandwich Place,Mediterranean Restaurant,Burrito Place
6,North York,Coffee Shop,Clothing Store,Restaurant,Japanese Restaurant,Bank,Pizza Place,Café,Park,Sandwich Place,Fast Food Restaurant
7,Scarborough,Chinese Restaurant,Bakery,Breakfast Spot,Bank,Coffee Shop,Fast Food Restaurant,Pharmacy,Convenience Store,Pizza Place,Playground
8,West Toronto,Café,Bar,Coffee Shop,Italian Restaurant,Restaurant,Pizza Place,Bakery,Breakfast Spot,Grocery Store,Park
9,York,Park,Convenience Store,Grocery Store,Trail,Restaurant,Bus Line,Sandwich Place,Hockey Arena,Field,Pool


In [46]:
# set number of clusters
kclusters = 5

grouped_clustering = grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]
merged = grouped

# add clustering labels
merged['Cluster Labels'] = kmeans.labels_

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
merged = merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

merged.head()

,Neighborhood,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Central Toronto,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.026316,...,Coffee Shop,Sandwich Place,Café,Park,Restaurant,Pizza Place,Sushi Restaurant,American Restaurant,Gym,Dessert Shop
1,Downtown Toronto,0.0,0.000862,0.000862,0.000862,0.000862,0.001724,0.002586,0.000862,0.014655,...,Coffee Shop,Café,Restaurant,Hotel,Japanese Restaurant,Italian Restaurant,Bakery,Park,Seafood Restaurant,Beer Bar
2,East Toronto,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.024000,...,Greek Restaurant,Coffee Shop,Italian Restaurant,Café,Brewery,Park,Restaurant,Ice Cream Shop,Bakery,Light Rail Station
3,East York,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,Coffee Shop,Bank,Pizza Place,Sporting Goods Shop,Pharmacy,Burger Joint,Park,Gym,Furniture / Home Store,Fast Food Restaurant
4,Etobicoke,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.012987,...,Pizza Place,Sandwich Place,Grocery Store,Coffee Shop,Fast Food Restaurant,Gym,Liquor Store,Pharmacy,Café,Pet Store
